In [1]:
!pip install pymystem3
!pip install transformers
!pip install umap-learn

In [2]:
import pandas as pd
# from razdel import tokenize
from nltk.corpus import stopwords
import nltk
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import re
from pymystem3 import Mystem
import numpy as np

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn.functional as f
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_data = pd.read_csv('/kaggle/input/1sem-project/anomaly-detection-competition-ml1-ts-autumn-2023/train_groups.csv')
docs_titles = pd.read_table('/kaggle/input/1sem-project/anomaly-detection-competition-ml1-ts-autumn-2023/docs_titles.tsv')

Добавим тайтлы в train_data

In [4]:
text_from_titles = []
pattern = re.compile(r'[^A-zА-я.,?!:;-]')
for idx in train_data['doc_id']:
    title = docs_titles[docs_titles['doc_id'] == idx]['title'].to_numpy()[0]
    try:
        title = re.sub(pattern, ' ', title)
        title = re.sub(' +', ' ', title)
        text_from_titles.append(title)
    
    except:
        text_from_titles.append('')

train_data['title'] = text_from_titles

**Bert Embeddings**

In [5]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")  #sberbank-ai/sbert_large_mt_nlu_ru
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [6]:
class Data(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = np.array(data)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):       
        text = self.data[idx]
        return text

In [7]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_embeddings(df, model, tokenizer, bsize=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    model.to(device)
    model.eval()    
    ldr_data = Data(data = df) 
    loader = DataLoader(dataset = ldr_data, shuffle=False, batch_size = min([bsize, len(df)]))
    embeddings = []
    with torch.no_grad():
        for i, inputs in tqdm(enumerate(loader)):
            ##Tokenize sentences
            batch = tokenizer(list(inputs), truncation=True, padding=True, return_tensors='pt', max_length=24)
            input_ids = batch['input_ids']
            input_ids = input_ids.to(device, dtype=torch.long)
            batch['input_ids'] = input_ids

            token_type_ids = batch['token_type_ids']
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            batch['token_type_ids'] = token_type_ids

            attention_mask = batch['attention_mask']
            attention_mask = attention_mask.to(device, dtype=torch.long)
            batch['attention_mask'] = attention_mask

            #Compute token embeddings
            preds = model(**batch)
            #Perform pooling. In this case, mean pooling
            preds = mean_pooling(preds, attention_mask)
            outputs = preds.cpu().detach().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)

In [8]:
# text_from_titles

In [9]:
text_from_titles

['ВАЗ Замена подшипников ступицы Нива',
 'Ваз оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 'Купить ступица Лада калина . Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 'Классика - ',
 'Ступица Нива замена подшипника своими руками',
 'ВАЗ ',
 'Обзор подшипников полуоси ВАЗ - , , ',
 'Купить Подшипники и ступицы FAG Страница ',
 'HorsePowers автомобильный интернет портал Отзыв владельца ВАЗ Нива года',
 'Новости и сообщения из официальной группы Вконтакте торговой компании Sport - Магазины - Тольятти',
 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 'Ступицы - OLX.ua - страница ',
 'маааленькая проблемка бортжурнал Автокам года на DRIVE ',
 'Разгруженные полуоси для Нива шлица мм ',
 'Прошивки для нива м . . скачать - Файлообменник emqraty .foxkirov.ru',
 'Страница . Раздела: Каталог Подвеска',
 'Продам Ниву ',
 'Рекомендации по проведению независимой экспертизы ОСАГО ',
 'Втулки подшипников в Омске. Сравнить це

In [10]:
text_embeddings = get_embeddings(text_from_titles, model, tokenizer, bsize=128)

cuda is used


92it [00:27,  3.31it/s]


In [11]:
text_embeddings.shape

(11690, 1024)

In [12]:
train_idxs = train_data[train_data['group_id'] > 30]['doc_id'].to_numpy()
test_idxs = train_data[train_data['group_id'] < 30]['doc_id'].to_numpy()

train_size = len(train_idxs)
test_size = len(test_idxs)

train_embeddings = {}

for i in range(train_size):
    train_embeddings[train_idxs[i]] = text_embeddings[i]

test_embeddings = {}

for i in range(test_size):
    test_embeddings[test_idxs[i]] = text_embeddings[train_size + i]

In [13]:
train_embeddings

{27066: array([ 0.17687666, -0.08629429,  0.12224551, ...,  0.17372091,
         0.4356556 ,  0.26140052], dtype=float32),
 26733: array([ 0.5596353 , -0.42228335, -1.1169655 , ..., -0.07868037,
         0.51552224, -0.04431295], dtype=float32),
 26386: array([-0.08155919, -0.10114788, -0.5477446 , ..., -0.26439488,
         0.4036506 , -0.17465033], dtype=float32),
 25385: array([ 0.6695799 , -0.18820354, -0.05292529, ..., -0.35368174,
        -0.10320017, -0.00247794], dtype=float32),
 27526: array([ 0.2144224 , -0.00807278,  0.45971683, ...,  0.35734758,
         0.19711412,  0.0558514 ], dtype=float32),
 25583: array([ 0.29647836, -0.40668124, -0.66013694, ...,  0.05734232,
        -0.05557977, -0.10266505], dtype=float32),
 26518: array([ 0.83008426, -0.14207476, -0.12726086, ...,  0.56855625,
         0.64111274, -0.01244617], dtype=float32),
 27779: array([ 0.24877243, -0.36109197, -0.34531736, ..., -0.11595857,
        -0.12134662,  0.04744019], dtype=float32),
 25337: array([0

In [14]:
from umap import UMAP

umap_3d = UMAP(n_components=3, init='random', random_state=0)
umap_3d.fit(text_embeddings[:train_size], train_data[train_data['group_id'] > 30]['target'].to_numpy())

/opt/conda/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


UMAP(init='random', n_components=3, random_state=0, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [15]:
import plotly.express as px

def get_3d_plot_for_group(data, embeddings, umap_obj, group_num):
    
    
    data_for_group = data[data['group_id'] == group_num]
    
    idxs = [idx for idx in data_for_group['doc_id']]
    
    texts = data_for_group['title']
    embeddings_plt = [embeddings[idx] for idx in idxs]
    
    proj_3d = umap_obj.transform(embeddings_plt)
    
    data_for_group['x'] = proj_3d[:, 0]
    data_for_group['y'] = proj_3d[:, 1]
    data_for_group['z'] = proj_3d[:, 2]
    
    fig_3d = px.scatter_3d(
        data_for_group, x='x', y='y', z='z',
        color='target', labels={'color', 'target'},
        hover_name='title'
    )
    
    return fig_3d

fig_3d = get_3d_plot_for_group(train_data, test_embeddings, umap_3d, 18)

fig_3d.show()

/opt/conda/lib/python3.10/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/tmp/ipykernel_42/926479743.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_group['x'] = proj_3d[:, 0]
/tmp/ipykernel_42/926479743.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_group['y'] = proj_3d[:, 1]
/tmp/ipykernel_42/926479743.py:17: SettingWithCopyWarning: 


**Обучение без дропа размерностей (1024 компонетный вектор)**

In [16]:
# for i in range(len(text_embeddings)):
#     train_data[f'comp_{i}'] = text_embeddings[:, i]

**С дропом размерностей**

In [17]:
test_dataset = train_data[train_complete['group_id'] > 110]
train_dataset = train_data[train_complete['group_id'] <= 110]

NameError: name 'train_complete' is not defined

In [ ]:
umap_60d = UMAP(n_components=500, init='random', random_state=0)
umap_60d.fit(text_embeddings[:len(train_dataset['target'])], train_dataset['target'].to_numpy())

In [ ]:
len(text_embeddings[:len(train_dataset['target'])])

In [ ]:
test_dataset

In [ ]:
test_embeddings_cut.shape

In [ ]:
train_size = len(train_dataset['target'])
test_size = len(test_dataset['target'])

train_embeddings_cut = umap_60d.transform(text_embeddings[:train_size])
test_embeddings_cut = umap_60d.transform(text_embeddings[train_size:train_size + test_size])

for i in range(train_embeddings_cut.shape[1]):
    train_dataset[f'comp_{i}'] = train_embeddings_cut[:, i]

for i in range(test_embeddings_cut.shape[1]):
    test_dataset[f'comp_{i}'] = test_embeddings_cut[:, i]

In [ ]:
X_train = train_dataset.drop(columns=['target', 'title'])
y_train = train_dataset['target']

X_test = test_dataset.drop(columns=['target', 'title'])
y_test = test_dataset['target']

In [ ]:
X_train

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

**GradientBoosting**

In [ ]:
clf = GradientBoostingClassifier(n_estimators=1000,
                                 learning_rate=0.05,
                                 verbose=1,
                                 random_state=0)
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

f1_score(y_test, preds)

**SVC**

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

umap_60d = UMAP(n_components=60, init='random', random_state=0)
umap_60d.fit(text_embeddings[:len(train_dataset['target'])], train_dataset['target'].to_numpy())

train_embeddings_cut = umap_60d.transform(text_embeddings[:train_size])
test_embeddings_cut = umap_60d.transform(text_embeddings[train_size:train_size + test_size])

# clf = SVC(gamma='auto', verbose=1)
# clf.fit(train_embeddings_cut, y_train)

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_embeddings_cut, y_train)

In [ ]:
preds = clf.predict(test_embeddings_cut)

f1_score(y_test, preds)